<a href="https://colab.research.google.com/github/pranukrish/CMPE255_DataMining/blob/main/Assignment3/ApacheBeamDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try Apache Beam - Python



# Setup

First, you need to set up your environment, installing `apache-beam` 

In [ ]:
!pip install --quiet apache-beam

#with beam.Pipeline() as pipeline:
#  pass  # build your pipeline here

     |████████████████████████████████| 10.9 MB 5.2 MB/s 
     |████████████████████████████████| 151 kB 70.7 MB/s 
     |████████████████████████████████| 508 kB 70.5 MB/s 
     |████████████████████████████████| 2.4 MB 56.8 MB/s 
     |████████████████████████████████| 270 kB 74.6 MB/s 
     |████████████████████████████████| 47 kB 5.4 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 1.0 MB 71.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220929150707 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
!pip install --quiet apache-beam[interactive]

# Mount drive to load file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read data into dataframe

In [ ]:
  df = beam.dataframe.io.read_csv('/content/drive/MyDrive/Data/EPL_Matches.csv')


# Create a pipeline

In [ ]:
import apache_beam as beam

p1 = beam.Pipeline()
visit_count = (
 
  p1
  |beam.dataframe.io.read_csv('/content/drive/MyDrive/Data/EPL_Matches.csv')

  |beam.Map(lambda record: record.split(';'))
  |beam.Filter(lambda record: record[3] == 'Arsenal')
  |beam.Map(lambda record: (record[4], 1))
  |beam.CombinePerKey(sum)
 
  |beam.io.WriteToText('/content/sample_data/out_data.txt')
)

p1.run()

# Implement Composite Transform

In [ ]:
class CustomTransform(beam.PTransform):
  def expand(self, input_coll):
 
  a = (
      input_coll
                      | 'Group and sum' >> beam.CombinePerKey(sum)
                      | 'filter Homegoals' >> beam.Filter(HomeGoals)
                      | 'Goals' >> beam.Map(format_output)
           
  )
  return a

def SplitRow(element):
  return element.split(',')

def filter_on_count(element):
 name, count = element
 if count > 1:
  return element
 def format_output(element):
 name, count = element
 #return ', '.join((name.encode('ascii'),str(count),'Regular employee'))
 return ', '.join((Home,str(count)))

p = beam.Pipeline()
input_collection = (

              p
                    | "Read from text file" >> beam.io.ReadFromText('goal_data.txt')
                    | "Split rows" >> beam.Map(SplitRow)
                  )

arsenal_count = (
                    input_collection
                    | 'Get all goals of Arsenal' >> beam.Filter(lambda record: record[3] == 'Arsenal')
                    | 'Pair with AwayGoals' >> beam.Map(lambda record: ("Arsenal, " +record[1], 1))
                    | 'Total goals made' >> CustomTransform()
                    | 'Write results for Arsenal' >> beam.io.WriteToText('arsenalgoals_output.txt')
                                    )

chelsea_count = (
              input_collection
              | 'Get all Chelsea goals' >> beam.Filter(lambda record: record[3] == 'Chelsea')
              | 'Pair with AwayGoals' >> beam.Map(lambda record: ("Chelsea, " +record[1], 1))
              | 'Total goals made' >> CustomTransform()
              | 'Write results for Chelsea' >> beam.io.WriteToText('chelseagoals_output.txt')
          )
p.run()